In [1]:
import re
import sys; sys.path.append("../")

import numpy as np
import pandas as pd

from lib.columns import rearrange_complaint_columns
from lib.explore import print_df
from datamatch import (
    ColumnsIndex, StringSimilarity, JaroWinklerSimilarity, DateSimilarity, ThresholdMatcher
)
from clean.baton_rouge_pd_cprr import clean_18
from clean.baton_rouge_csd_pprr import clean_17 as clean_csd_17, clean_19 as clean_csd_19

In [2]:
# df17 = clean_csd_17()
# df17

In [3]:
# df19 = clean_csd_19()
# df19

In [4]:
df17 = pd.read_csv('../data/match/pprr_baton_rouge_csd_2017.csv')
df19 = pd.read_csv('../data/match/pprr_baton_rouge_csd_2019.csv')
cprr = pd.read_csv('../data/match/cprr_baton_rouge_pd_2018.csv')

In [13]:
cprr

,investigation_status,action,disposition,department_code,department_desc,last_name,first_name,middle_initial,middle_name,receive_year,...,receive_day,occur_year,occur_month,occur_day,tracking_number,charges,agency,data_production_year,uid,charge_uid
0,administrative review,exonerated,exonerated,P1254,cib street crimes unit,barcelona,jesse,r,NaN,2018,...,2,2017.0,12.0,30.0,2018-002,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2017,12f45a233bb822ea595c0dabc7e92d7d,c7a0a3a7160e52f47b9965b7e8c82bd5
1,administrative review,not sustained,not sustained,P10280,cib narcotics,acree,jason,k,NaN,2018,...,9,2018.0,1.0,9.0,2018-004,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018,7dd3a1e5b411318feab180ce8d0eb8b9,9de8007a01ffb4d68a4c2fd186a4a9bd
2,office investigation,resigned,office investigation,P10476,op service comm center,rimes,melissa,j,NaN,2018,...,9,2017.0,12.0,3.0,2018-003,2:4 abuse of sick leave - 66,Baton Rouge PD,2017,2f584d20925f6afe3fcb930ed908e5f7,cbd6e6b11930cb067194fdc4d290b4f2
3,administrative review,not sustained,not sustained,P10578,cib cib,hamadeh,yuseff,g,NaN,2018,...,10,2018.0,1.0,10.0,2018-005,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2018,ef3a5a4eb4544ebeb4199f9fe1016105,87ed457ca0d91a317754b43f530c3223
4,administrative review,not sustained,not sustained,P10252,patrol 2nd district,pollar jr,charles,g,NaN,2018,...,11,2018.0,1.0,11.0,2018-006,3:20 use of force - 53 hard empty hand,Baton Rouge PD,2018,3973b0e701da4fef807b4d2abf143871,c59024deb7e2dac3383d0462126b5ed6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,NaN,2019,...,30,2018.0,3.0,27.0,2019-003,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018,c4994d8353864109fca8fe4c881bf20f,944cf18ddacafffe023c3840d07f5ddc
137,administrative review,letter of reprimand,not sustained,P1245,patrol 2nd district,clark,kenneth,m,NaN,2019,...,30,2018.0,3.0,27.0,2019-003,3:18 insubordination - 43,Baton Rouge PD,2018,c4994d8353864109fca8fe4c881bf20f,e1e04016333fccc3a02a6ca693e81bf8
138,administrative review,letter of reprimand,sustained,P1245,patrol 2nd district,clark,kenneth,m,NaN,2019,...,30,2018.0,3.0,27.0,2019-003,2:12 conduct unbecoming an officer - 21,Baton Rouge PD,2018,c4994d8353864109fca8fe4c881bf20f,fe3f1097cdf1086834f282005ce4731d
139,administrative review,not sustained,not sustained,P10639,patrol 1st district,robertson,jason,r,NaN,2018,...,26,2018.0,1.0,26.0,2018-010,3:17 carrying out orders - 40 general orders (...,Baton Rouge PD,2018,f0098284466343cf5316ac41763d2548,e05b5c9f2420bb255ef8f28db874ccdf


In [16]:
cprr.loc[cprr.action.fillna('').str.contains('termina'), ['receive_year', 'receive_month', 'receive_day', 'uid']]

,receive_year,receive_month,receive_day,uid
59,2018,8,7,ef3a5a4eb4544ebeb4199f9fe1016105
61,2018,8,7,ef3a5a4eb4544ebeb4199f9fe1016105
66,2018,8,7,ef3a5a4eb4544ebeb4199f9fe1016105
67,2018,8,7,ef3a5a4eb4544ebeb4199f9fe1016105
68,2018,8,7,ef3a5a4eb4544ebeb4199f9fe1016105
69,2018,8,7,ef3a5a4eb4544ebeb4199f9fe1016105


In [8]:
df17[df17.uid == 'ef3a5a4eb4544ebeb4199f9fe1016105']

,year,employee_id,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,...,resign_month,resign_day,agency,uid,pay_effective_year,rank_year,dept_year,perhist_uid,level_1_cert_date,last_pc_12_qualification_date
3204,2016,524689,hamadeh,yuseff,g,5020001,police department-uniform patrol bureau,5005,police officer,0,...,NaN,NaN,Baton Rouge CSD,ef3a5a4eb4544ebeb4199f9fe1016105,2016.0,2016.0,2016.0,cddf4bb54de1aebd414cba9b352e3edd,NaN,NaN
3205,2017,524689,hamadeh,yuseff,g,5060001,police department-criminal invest bureau,5005,police officer,0,...,NaN,NaN,Baton Rouge CSD,ef3a5a4eb4544ebeb4199f9fe1016105,2017.0,NaN,2017.0,c0bad26bcf22757316276703b10bf361,NaN,NaN


In [10]:
df19.loc[df19.uid == 'ef3a5a4eb4544ebeb4199f9fe1016105', ['resign_year', 'resign_month', 'resign_day']]

,resign_year,resign_month,resign_day
1854,2019.0,2.0,14.0
1855,2019.0,2.0,14.0
